# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [18]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [19]:
# Memuat data hasil clustering
dataframe = pd.read_csv('Clustered_Customers.csv')

In [20]:
# Seleksi hanya kolom numerik untuk analisis lebih lanjut
numeric_columns = dataframe.select_dtypes(include=['float64', 'int64']).columns
dataframe_numeric = dataframe[numeric_columns]

# Menambahkan kembali kolom 'ClusterGroup' jika diperlukan
if 'ClusterGroup' in dataframe.columns:
    dataframe_numeric['ClusterGroup'] = dataframe['ClusterGroup']

print(dataframe.head())

   CustomerID  Gender       Age  Annual Income ($)  Spending Score (1-100)  \
0           1  Doctor -1.052345          -2.086675               -0.431850   
1           2  Doctor -0.981941          -1.650205                1.069755   
2           3  Artist -1.017143          -0.537207               -1.611684   
3           4  Artist -0.911538          -1.126441                0.926745   
4           5  Artist -0.629924          -1.584734               -0.396098   

      Profession  Work Experience  Family Size  ClusterGroup  
0     Healthcare                1            4             2  
1       Engineer                3            3             2  
2       Engineer                1            1             2  
3         Lawyer                0            2             2  
4  Entertainment                2            6             2  


<ipython-input-20-5bd7b4d18200>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe_numeric['ClusterGroup'] = dataframe['ClusterGroup']


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [21]:
# Pastikan 'ClusterGroup' digunakan sebagai label
data_features = dataframe_numeric.drop(columns=['ClusterGroup'], errors='ignore')

data_labels = dataframe['ClusterGroup']
features_train, features_test, labels_train, labels_test = train_test_split(
    data_features, data_labels, test_size=0.2, random_state=42
)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [22]:
classifiers = {
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
}

results_summary = {}

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

Analisis Penggunaan Algoritma:

1. Decision Tree digunakan sebagai baseline karena sederhana dan mudah diinterpretasikan.
2. Random Forest digunakan untuk meningkatkan akurasi dan mengurangi risiko overfitting dengan prinsip ensemble.
3. KNN dipilih karena efektif untuk data sederhana, meskipun membutuhkan preprocessing tambahan seperti standarisasi fitur.
Pemilihan algoritma ini memungkinkan perbandingan antara model berbasis pohon (Decision Tree dan Random Forest) dengan model berbasis jarak (KNN), sehingga memberikan pandangan yang lebih luas terhadap performa pada dataset.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [23]:
for model_name, classifier in classifiers.items():
    # Melatih model
    classifier.fit(features_train, labels_train)
    predictions = classifier.predict(features_test)

    # Evaluasi model
    accuracy = accuracy_score(labels_test, predictions)
    precision = precision_score(labels_test, predictions, average='weighted')
    recall = recall_score(labels_test, predictions, average='weighted')
    f1 = f1_score(labels_test, predictions, average='weighted')
    confusion_mat = confusion_matrix(labels_test, predictions)

    results_summary[model_name] = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "ConfusionMatrix": confusion_mat,
    }

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

#### Identifikasi Model
**Precision, Recall, dan F1-Score:** Semua metrik menunjukkan hasil sempurna (1.0000) untuk ketiga algoritma, yang berarti tidak ada kesalahan klasifikasi pada data uji.  
**Confusion Matrix:** Tidak ada prediksi yang keliru, dan setiap kelas teridentifikasi dengan akurat. Namun, hasil ini bisa menandakan adanya masalah potensial, seperti:
- Dataset yang terlalu sederhana atau cluster yang sangat terpisah, sehingga model tidak diuji dalam kondisi yang lebih kompleks.
- Kebocoran data: Jika informasi dari data uji secara tidak langsung masuk ke data latih, model dapat menunjukkan performa yang sempurna namun tidak realistis.

#### Analisis Fitting
**Overfitting:** Model menunjukkan performa sempurna pada data uji, tetapi ini bisa berarti model terlalu "mengingat" data daripada memahami pola yang sebenarnya. Jika dataset mengandung noise atau kurang bervariasi, risiko overfitting meningkat.  
**Underfitting:** Tidak terdeteksi karena metrik evaluasi yang sempurna, yang menunjukkan bahwa model mampu menangkap pola dalam data dengan sangat baik (atau bahkan terlalu baik).

#### Rekomendasi Tindakan Lanjutan
1. **Validasi Model dengan Cross-Validation:** Terapkan k-fold cross-validation untuk menguji kemampuan generalisasi model pada seluruh dataset. Ini membantu memastikan bahwa performa model konsisten dan tidak hanya baik pada satu subset data.
2. **Analisis Dataset:** Pastikan tidak ada kebocoran data, seperti fitur yang terlalu berkorelasi langsung dengan label (kolom Cluster). Periksa distribusi data untuk setiap fitur dan kelas agar dataset mencerminkan kondisi dunia nyata.
3. **Menggunakan Dataset yang Lebih Kompleks:** Jika dataset saat ini terlalu sederhana atau tersegmentasi, pertimbangkan untuk mengumpulkan data tambahan yang lebih bervariasi dan kompleks.
4. **Eksperimen dengan Algoritma Lain:** Cobalah algoritma lain seperti Logistic Regression, Support Vector Machines (SVM), atau Gradient Boosting (misalnya, XGBoost, LightGBM) untuk membandingkan performa.
5. **Tuning Hyperparameter:** Lakukan pencarian hyperparameter untuk model seperti Random Forest atau KNN untuk memastikan hasil yang optimal.
6. **Evaluasi dengan Data Baru:** Jika memungkinkan, uji model dengan dataset baru untuk menilai kemampuannya dalam menangani data yang belum pernah dilihat sebelumnya.